# Data Cleaning – Online Retail Dataset

## Objective
The goal of this notebook is to clean and validate the Online Retail transactional data
to make it suitable for downstream feature engineering and recommendation modeling.

This includes:
- Removing cancelled and returned transactions
- Handling invalid quantities and prices
- Standardizing data types
- Producing a clean, reproducible dataset

No modeling or feature engineering is performed here.


In [1]:
import pandas as pd
import numpy as np

from pathlib import Path

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)


In [2]:
BASE_DIR = Path().resolve().parent
DATA_FILE = BASE_DIR / "data" / "data.csv"

print("Loading raw data from:", DATA_FILE)

df = pd.read_csv(
    DATA_FILE,
    encoding="ISO-8859-1"
)

Loading raw data from: D:\Code Playground\ML_Ops\product-bundle-recommender-system\data\data.csv


In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.00,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.00,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.00,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.00,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.00,United Kingdom


In [4]:
print("Initial shape:", df.shape)

df.isna().sum()


Initial shape: (541909, 8)


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## Handling Cancelled Transactions

Invoice numbers starting with 'C' indicate cancelled or refunded orders.
These transactions do not represent successful purchases and must be removed.


In [5]:
cancelled_mask = df["InvoiceNo"].astype(str).str.startswith("C")

print("Cancelled transactions:", cancelled_mask.sum())

df = df[~cancelled_mask]

print("Shape after removing cancellations:", df.shape)

Cancelled transactions: 9288
Shape after removing cancellations: (532621, 8)


## Handling Invalid Quantities

Negative or zero quantities typically indicate returns, corrections,
or invalid transactions. These rows are removed.


In [6]:
invalid_qty = df["Quantity"] <= 0

print("Invalid quantity rows:", invalid_qty.sum())

df = df[~invalid_qty]

print("Shape after quantity filtering:", df.shape)

Invalid quantity rows: 1336
Shape after quantity filtering: (531285, 8)


## Handling Invalid Prices

Rows with zero or negative unit prices do not represent valid purchases
and are removed.


In [7]:
invalid_price = df["UnitPrice"] <= 0

print("Invalid price rows:", invalid_price.sum())

df = df[~invalid_price]

print("Shape after price filtering:", df.shape)

Invalid price rows: 1181
Shape after price filtering: (530104, 8)


## Handling Missing CustomerID

CustomerID is missing for a subset of transactions.
Since this project focuses on product bundle recommendation (not personalization),
these rows are retained.


In [8]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
df["CustomerID"] = df["CustomerID"].astype("Int64")

In [9]:
critical_cols = ["InvoiceNo", "StockCode", "InvoiceDate"]

before = df.shape[0]

df = df.dropna(subset=critical_cols)

after = df.shape[0]

print(f"Removed {before - after} rows with missing critical fields")

Removed 0 rows with missing critical fields


## Cleaning Product Descriptions

Minor text normalization is applied for consistency in reporting.
This does not affect modeling logic.


In [10]:
df["Description"] = (
    df["Description"]
    .astype(str)
    .str.strip()
    .str.upper()
)

In [11]:
print("Final shape after cleaning:", df.shape)

df.describe(include="number").T


Final shape after cleaning: (530104, 8)


,count,mean,std,min,25%,50%,75%,max
Quantity,530104.00,10.54,155.52,1.00,1.00,3.00,10.00,80995.00
UnitPrice,530104.00,3.91,35.92,0.00,1.25,2.08,4.13,13541.33
CustomerID,397884.00,15294.42,1713.14,12346.00,13969.00,15159.00,16795.00,18287.00


In [12]:
basket_sizes = df.groupby("InvoiceNo")["StockCode"].count()

basket_sizes.describe()


count   19960.00
mean       26.56
std        47.54
min         1.00
25%         6.00
50%        15.00
75%        29.00
max      1114.00
Name: StockCode, dtype: float64

## Handling Missing Product Descriptions

Some transactions have missing product descriptions.
Since StockCode uniquely identifies products and is used for modeling,
these rows are retained and missing descriptions are filled with a placeholder.


In [13]:
df["Description"] = df["Description"].fillna("UNKNOWN_PRODUCT")

In [14]:
OUTPUT_PATH = BASE_DIR / "data" / "processed"
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

clean_file = OUTPUT_PATH / "clean_transactions.parquet"

df.to_parquet(clean_file, index=False)

print("Clean data saved to:", clean_file)

Clean data saved to: D:\Code Playground\ML_Ops\product-bundle-recommender-system\data\processed\clean_transactions.parquet


## Cleaning Summary

- Cancelled and refunded transactions removed
- Invalid quantities and prices filtered
- Data types standardized
- Missing CustomerID retained for bundle-focused modeling
- Clean transactional dataset saved for downstream processing

The dataset is now suitable for feature engineering and recommendation modeling.
